In [ ]:
! pip uninstall -y langchain langchain-core langchain-community langchain-openai langgraph \
  openai faiss-cpu tiktoken pydantic packaging numpy

Found existing installation: langchain 0.3.7
Uninstalling langchain-0.3.7:
  Successfully uninstalled langchain-0.3.7
Found existing installation: langchain-core 0.3.18
Uninstalling langchain-core-0.3.18:
  Successfully uninstalled langchain-core-0.3.18
Found existing installation: langchain-community 0.3.7
Uninstalling langchain-community-0.3.7:
  Successfully uninstalled langchain-community-0.3.7
Found existing installation: langchain-openai 0.2.7
Uninstalling langchain-openai-0.2.7:
  Successfully uninstalled langchain-openai-0.2.7
Found existing installation: langgraph 0.2.39
Uninstalling langgraph-0.2.39:
  Successfully uninstalled langgraph-0.2.39
Found existing installation: openai 1.54.0
Uninstalling openai-1.54.0:
  Successfully uninstalled openai-1.54.0
Found existing installation: faiss-cpu 1.8.0
Uninstalling faiss-cpu-1.8.0:
  Successfully uninstalled faiss-cpu-1.8.0
Found existing installation: tiktoken 0.7.0
Uninstalling tiktoken-0.7.0:
  Successfully uninstalled tiktoken

In [ ]:
!pip install -U \
  "langchain==0.3.27" \
  "langchain-core==0.3.74" \
  "langchain-community==0.3.27" \
  "langchain-openai==0.3.31" \
  "langgraph==0.2.45" \
  "openai==1.100.0" \
  "faiss-cpu==1.12.0" \
  "tiktoken==0.11.0" \
  "pypdf==5.1.0" \
  "tqdm==4.67.1" \
  "python-dotenv==1.0.1" \
  "pydantic==2.11.7" \
  "numpy==2.0.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 5.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.5/786.5 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# 코랩 환경에서 실행 예시
# ========================
# 환경 변수 설정
import os
os.environ["OPENAI_API_KEY"] =  "fill out your API key"

In [ ]:
# =============================================================================
# 📌 LangGraph RAG (소비자분쟁) - 최종 합본 (문단형/미괄식 요약)
# 흐름: 질문 → 라우팅(질문판단기) → 해결기준 + 법률 → (서비스/상품)사례
#      → 컴바인(문단 초안) → 한문단 요약(미괄식) → 결과
# =============================================================================
# Colab 설치 예시:
# !pip install -q langchain langchain-openai langchain-community langgraph faiss-cpu openai tiktoken

from __future__ import annotations
import os, re
from typing import TypedDict, List, Annotated, Dict, Any

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver

# -----------------------------------------------------------------------------
# 환경 설정 (API 키/모델)
# -----------------------------------------------------------------------------
if not os.environ.get("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY를 먼저 설정하세요. 예) os.environ['OPENAI_API_KEY']='sk-...'")

LLM = ChatOpenAI(model="gpt-4o-mini", temperature=0)
EMB = OpenAIEmbeddings(model="text-embedding-3-large")

# -----------------------------------------------------------------------------
# 벡터 DB 경로
# -----------------------------------------------------------------------------
DB_GUIDELINE = "fill out DB path"
DB_LAW       = "fill out DB path"
DB_CASE_SVC  = "fill out DB path"
DB_CASE_PRD  = "fill out DB path"

VG     = FAISS.load_local(DB_GUIDELINE, EMB, allow_dangerous_deserialization=True)
VL     = FAISS.load_local(DB_LAW,       EMB, allow_dangerous_deserialization=True)
VCSVC  = FAISS.load_local(DB_CASE_SVC,  EMB, allow_dangerous_deserialization=True)
VCPRD  = FAISS.load_local(DB_CASE_PRD,  EMB, allow_dangerous_deserialization=True)

# -----------------------------------------------------------------------------
# 인스트럭션 + 프롬프트
# -----------------------------------------------------------------------------
BASE_INSTRUCTION = (
    "당신은 소비자분쟁 전문가입니다. 주어진 질문의 요지를 파악하고 적절한 대답을 해주세요.\n"
    "주어진 질문에 대해서 사전에 반드시 주어진 문서만을 토대로 기준 지침을 파악한 후, "
    "법률적 조언이 필요한 대답은 제시된 '민법', '소비자보호법', '전자상거래법'의 문서를 참조하여 답변을 구성해주세요.\n"
    "만약 사용자가 사례를 요청할 시, 제시된 '분쟁조정사례' 문서를 참조하여 사례의 구체적 내용을 함께 말해주세요.\n"
    "적절한 답변이 불가능할 시 'I don't know'로 대답해 주세요.\n\n"
    "추상적 단어의 처리방법\n"
    " - 비공개 장소: 화장실, 개인 객실 등\n"
    " - 민감정보: 개인 주민등록번호, 주소, 연락처 등\n"
    "위와 같이 질문의 추상적인 단어는 너가 보유한 지식체계 내에서 구체화하여 고려한 다음에 답변을 부탁해."
)

GUIDELINE_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        f"{BASE_INSTRUCTION}\n"
        "다음은 '소비자분쟁 해결기준(개정안)' 문서 발췌입니다:\n\n"
        "{context}\n\n"
        "질문: {question}\n\n"
        "→ (1) 처리원칙/책임주체, (2) 교환·환불·수리 등 보상기준, "
        "(3) 증빙/절차, (4) 예외·유의사항을 항목별로 제시하세요."
    ),
)

LAW_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        f"{BASE_INSTRUCTION}\n"
        "법률 범위는 민법, 소비자기본법, 전자상거래법으로 한정합니다.\n"
        "다음은 법률/시행령 문서 발췌입니다:\n\n"
        "{context}\n\n"
        "질문: {question}\n\n"
        "출력 형식(반드시 준수):\n"
        "[적용 법조문]\n"
        "- 법률명 제n조 제m항: 요지\n"
        "- (필요 시) 법률명 제p조: 요지\n"
        "[분석]\n"
        "- 요건/의무/금지\n"
        "- 위반 시 조치(과태료/손해배상 등)\n\n"
        "주의: 최소 1개 이상의 정확한 조문 표기를 포함하고, 문서에 없는 조문을 추정하지 마세요."
    ),
)

CASE_PROMPT = PromptTemplate(
    input_variables=["context", "question", "category"],
    template=(
        f"{BASE_INSTRUCTION}\n"
        "다음은 '{category}' 카테고리의 소비자분쟁 사례 발췌입니다:\n\n"
        "{context}\n\n"
        "질문: {question}\n\n"
        "→ (1) 사실관계, (2) 쟁점, (3) 판단/조정결과, (4) 적용 포인트를 간결히 요약하세요.\n"
        "※ 반드시 '**한국소비자원 제공 사례에 따르면**'으로 시작하세요."
    ),
)

# -----------------------------------------------------------------------------
# 3) 상태 정의
# -----------------------------------------------------------------------------
class QAState(TypedDict):
    query: Annotated[str, "query"]
    need_guideline: bool
    need_law: bool
    need_case_service: bool
    need_case_product: bool
    result_guideline: str
    result_law: str
    result_case_service: str
    result_case_product: str
    law_citations: List[str]
    combined: str
    summary: str
    source_documents_guideline: List
    source_documents_law: List
    source_documents_case_service: List
    source_documents_case_product: List
    source_documents: List

# -----------------------------------------------------------------------------
# 리트리버 & 체인
# -----------------------------------------------------------------------------
ret_gdl = VG.as_retriever(search_type="similarity", search_kwargs={"k": 2, "fetch_k": 7})
ret_law = VL.as_retriever(search_type="similarity", search_kwargs={"k": 2, "fetch_k": 7})
ret_csv = VCSVC.as_retriever(search_type="similarity", search_kwargs={"k": 2, "fetch_k": 7})
ret_cpr = VCPRD.as_retriever(search_type="similarity", search_kwargs={"k": 2, "fetch_k": 7})

chain_gdl = RetrievalQA.from_chain_type(
    llm=LLM, retriever=ret_gdl, return_source_documents=True,
    chain_type="stuff", chain_type_kwargs={"prompt": GUIDELINE_PROMPT},
)
chain_law = RetrievalQA.from_chain_type(
    llm=LLM, retriever=ret_law, return_source_documents=True,
    chain_type="stuff", chain_type_kwargs={"prompt": LAW_PROMPT},
)
chain_csv = RetrievalQA.from_chain_type(
    llm=LLM, retriever=ret_csv, return_source_documents=True,
    chain_type="stuff", chain_type_kwargs={"prompt": CASE_PROMPT.partial(category="서비스")},
)
chain_cpr = RetrievalQA.from_chain_type(
    llm=LLM, retriever=ret_cpr, return_source_documents=True,
    chain_type="stuff", chain_type_kwargs={"prompt": CASE_PROMPT.partial(category="상품")},
)

# -----------------------------------------------------------------------------
# 조문 자동 추출
# -----------------------------------------------------------------------------
COMBINED_ARTICLE_RE = re.compile(
    r"(제\s*\d+\s*조(?:\s*의\s*\d+)?)"
    r"(?:\s*(?:[,·]|\s)?\s*제\s*(\d+)\s*항)?",
    re.UNICODE
)

def extract_statute_citations(source_docs: List) -> List[str]:
    found: List[str] = []
    seen: set = set()
    for d in source_docs or []:
        text = getattr(d, "page_content", "") or ""
        meta = getattr(d, "metadata", {}) or {}
        law_name = (meta.get("law_name") or "").strip()

        for m in COMBINED_ARTICLE_RE.finditer(text):
            article = (m.group(1) or "").strip()
            clause  = (m.group(2) or "").strip()
            clause_str = f" 제{clause}항" if clause else ""
            prefix = f"{law_name} " if law_name else ""
            citation = f"{prefix}{article}{clause_str}".replace("  ", " ")
            if citation and citation not in seen:
                seen.add(citation)
                found.append(citation)
                if len(found) >= 8:
                    return found
    return found

# -----------------------------------------------------------------------------
# 질문 판단기(라우팅)
# -----------------------------------------------------------------------------
OVERRIDE_BOTH   = ["[both]", "[종합]", "[all]"]
OVERRIDE_LAW    = ["[법]", "[law]", "[조문]", "[민법]", "[소비자법]", "[전자상거래법]", "[법리]", "[legal]"]
OVERRIDE_CASE   = ["[사례]", "[case]", "[precedent]"]
OVERRIDE_SVC    = ["[서비스]", "[service]"]
OVERRIDE_PRD    = ["[상품]", "[product]"]
OVERRIDE_GDL    = ["[기준]", "[guideline]", "[해결기준]"]

NEG_CASE_TAGS   = ["[nocase]", "[case_off]"]
NEG_LAW_TAGS    = ["[nolaw]", "[law_off]"]
NEG_GDL_TAGS    = ["[noguideline]", "[gdl_off]"]
NO_CASE_PHRASE  = ["사례는 빼", "사례 빼고", "사례 제외", "유사사례 제외"]
NO_LAW_PHRASE   = ["법률은 빼", "법은 빼고", "조문 제외", "법리 제외"]
NO_GDL_PHRASE   = ["해결기준은 빼", "기준 제외", "기준은 빼고"]

BOTH_HINTS = ["법률과 사례 모두", "법과 사례 모두", "둘 다", "종합 검토", "함께 검토", "동시에", "비교해서", "종합해서"]
GDL_HINTS  = ["보상기준","교환","환불","수리","AS","청약철회","반품","취소","위약금","수수료","지연배상",
              "처리원칙","책임주체","절차","증빙","기간","기한","예외","유의사항"]
LAW_HINTS  = ["법률","법령","조문","근거","시행령","시행규칙","규정","벌칙","과태료","민사","형사",
              "손해배상","배상책임","입증책임","소멸시효","제척기간","관할","약관","불공정약관",
              "민법","소비자기본법","전자상거래법","전자상거래 등에서의 소비자보호"]
LAW_STRICT = ["법리","법률적으로","법률 위주","법적 검토","법적 판단","조문 중심","법률 검토"]
CASE_HINTS = ["유사사례","사례","분쟁사례","판정","조정","판례","결정","사실관계 유사"]

SERVICE_HINTS = ["여행","항공","항공권","항공기","지연","결항","숙박","호텔","택배","배송지연","운송",
                 "수강","학원","강의","구독","멤버십","통신","인터넷","설치","시공","시술","미용","수리","용역","플랫폼"]
PRODUCT_HINTS = ["상품","제품","가전","전자제품","휴대폰","노트북","TV","냉장고","세탁기","의류","신발",
                 "식품","화장품","가구","유아용품","완구","하자","불량","초기불량","부품","교환요청"]

def _has_any(tokens: List[str], text: str) -> bool:
    return any(tok in text for tok in tokens)

def classify_needs(q: str) -> Dict[str, bool]:
    q = (q or "").strip()
    need_guideline = True
    need_law = False
    need_case_service = False
    need_case_product = False

    # 배제
    if _has_any(NEG_GDL_TAGS, q) or _has_any(NO_GDL_PHRASE, q): need_guideline = False
    if _has_any(NEG_LAW_TAGS, q) or _has_any(NO_LAW_PHRASE, q): need_law = False
    if _has_any(NEG_CASE_TAGS, q) or _has_any(NO_CASE_PHRASE, q):
        need_case_service = need_case_product = False

    # 명시
    if _has_any(OVERRIDE_BOTH, q) or _has_any(BOTH_HINTS, q):
        need_law = True
        if _has_any(OVERRIDE_SVC + SERVICE_HINTS, q): need_case_service = True
        if _has_any(OVERRIDE_PRD + PRODUCT_HINTS, q): need_case_product = True
        if not (need_case_service or need_case_product): need_case_service = True
        return {"need_guideline": need_guideline, "need_law": need_law,
                "need_case_service": need_case_service, "need_case_product": need_case_product}

    if _has_any(OVERRIDE_GDL, q) and not (_has_any(OVERRIDE_LAW, q) or _has_any(OVERRIDE_CASE, q)):
        need_law = need_case_service = need_case_product = False
        return {"need_guideline": need_guideline, "need_law": need_law,
                "need_case_service": need_case_service, "need_case_product": need_case_product}

    if _has_any(OVERRIDE_LAW, q):
        need_law = True
        if _has_any(OVERRIDE_SVC, q): need_case_service = True
        if _has_any(OVERRIDE_PRD, q): need_case_product = True
        return {"need_guideline": need_guideline, "need_law": need_law,
                "need_case_service": need_case_service, "need_case_product": need_case_product}

    if _has_any(OVERRIDE_CASE, q):
        if _has_any(OVERRIDE_SVC + SERVICE_HINTS, q): need_case_service = True
        if _has_any(OVERRIDE_PRD + PRODUCT_HINTS, q): need_case_product = True
        if not (need_case_service or need_case_product): need_case_service = True
        need_law = False
        return {"need_guideline": need_guideline, "need_law": need_law,
                "need_case_service": need_case_service, "need_case_product": need_case_product}

    # 자연어 의도
    if _has_any(LAW_STRICT, q):
        need_law = True
        if _has_any(CASE_HINTS, q):
            if _has_any(SERVICE_HINTS, q): need_case_service = True
            elif _has_any(PRODUCT_HINTS, q): need_case_product = True
        else:
            need_case_service = need_case_product = False
        return {"need_guideline": need_guideline, "need_law": need_law,
                "need_case_service": need_case_service, "need_case_product": need_case_product}

    if _has_any(GDL_HINTS, q):
        if _has_any(LAW_HINTS, q): need_law = True

    if _has_any(CASE_HINTS, q):
        if _has_any(SERVICE_HINTS, q): need_case_service = True
        elif _has_any(PRODUCT_HINTS, q): need_case_product = True
        else: need_case_service = True
        if _has_any(LAW_HINTS, q): need_law = True

    # 기본 폴백
    if not (need_law or need_case_service or need_case_product):
        need_law = True
        if _has_any(SERVICE_HINTS, q): need_case_service = True
        elif _has_any(PRODUCT_HINTS, q): need_case_product = True

    # 최종 배제 재적용
    if _has_any(NEG_LAW_TAGS, q) or _has_any(NO_LAW_PHRASE, q): need_law = False
    if _has_any(NEG_CASE_TAGS, q) or _has_any(NO_CASE_PHRASE, q):
        need_case_service = need_case_product = False
    if _has_any(NEG_GDL_TAGS, q) or _has_any(NO_GDL_PHRASE, q): need_guideline = False

    return {"need_guideline": need_guideline, "need_law": need_law,
            "need_case_service": need_case_service, "need_case_product": need_case_product}

# -----------------------------------------------------------------------------
# 노드들
# -----------------------------------------------------------------------------
def router_node(state: QAState) -> QAState:
    return {**state, **classify_needs(state["query"])}

def guideline_node(state: QAState) -> QAState:
    if not state.get("need_guideline", True):
        return {**state, "result_guideline": "", "source_documents_guideline": []}
    r = chain_gdl.invoke({"query": state["query"]})
    return {**state, "result_guideline": r["result"], "source_documents_guideline": r.get("source_documents", [])}

def law_node(state: QAState) -> QAState:
    if not state.get("need_law", False):
        return {**state, "result_law": "", "source_documents_law": [], "law_citations": []}
    r = chain_law.invoke({"query": state["query"]})
    cites = extract_statute_citations(r.get("source_documents", []))
    return {
        **state,
        "result_law": r["result"],
        "source_documents_law": r.get("source_documents", []),
        "law_citations": cites,
    }

def case_service_node(state: QAState) -> QAState:
    if not state.get("need_case_service", False):
        return {**state, "result_case_service": "", "source_documents_case_service": []}
    r = chain_csv.invoke({"query": state["query"]})
    return {**state, "result_case_service": r["result"], "source_documents_case_service": r.get("source_documents", [])}

def case_product_node(state: QAState) -> QAState:
    if not state.get("need_case_product", False):
        return {**state, "result_case_product": "", "source_documents_case_product": []}
    r = chain_cpr.invoke({"query": state["query"]})
    return {**state, "result_case_product": r["result"], "source_documents_case_product": r.get("source_documents", [])}

# ---------- 컴바인 ----------
def combine_node(state: QAState) -> QAState:
    parts = []
    if state.get("result_guideline"):     parts.append(state["result_guideline"])
    if state.get("result_law"):           parts.append(state["result_law"])
    if state.get("result_case_service"):  parts.append(state["result_case_service"])
    if state.get("result_case_product"):  parts.append(state["result_case_product"])

    cites = state.get("law_citations") or []
    cites_line = ", ".join(cites)

    if not parts:
        combined_text = "I don't know"
    else:
        material = "\n\n".join(parts)
        prompt = (
            "아래 자료만을 근거로, 소비자분쟁 해결기준과 관련 법률, 그리고 (있다면) 유사사례를 "
            "자연스러운 한국어 문단으로 통합 요약하라. 섹션 라벨(예: '법률:' 등)은 쓰지 말고, "
            "최대 두 문단으로 간결하게 정리하라. 사실/수치/조문은 자료에 있는 것만 쓰고 추정 금지.\n"
            f"{('참고 조문: ' + cites_line) if cites_line else ''}\n\n"
            "자료:\n" + material
        )
        combined_text = LLM.invoke(prompt).content

    all_sources = []
    all_sources += state.get("source_documents_guideline", [])
    all_sources += state.get("source_documents_law", [])
    all_sources += state.get("source_documents_case_service", [])
    all_sources += state.get("source_documents_case_product", [])
    return {**state, "combined": combined_text, "source_documents": all_sources}

# ---------- 한 문단 요약 ----------
def summarize_text_prose(text: str, *,
                         have_guideline: bool,
                         have_law: bool,
                         have_cases: bool,
                         citations: List[str]) -> str:
    cite_str = ", ".join(citations) if citations else ""
    prompt = (
        "아래 통합 초안만을 근거로, 최초에 제시된 질문의 요지에 맞게 한국어로 한 문단의 자연스러운 의견서를 작성하라.\n"
        "요구사항:\n"
        "1) 섹션 라벨/불릿/번호 금지, 완전한 문장들로만 구성\n"
        "2) 존재하지 않는 정보(법조문 번호, 사례 요지, 수치)는 절대 추정/창작하지 말고 언급하지 말 것\n"
        "3) 법조문이 있으면 문장 중에 자연스럽게 괄호 등으로 녹여 쓰되, 없으면 법조문 언급 자체를 생략\n"
        "4) 사례가 있으면 '유사한 분쟁조정 사례에 비추어' 수준으로 자연스럽게 녹여 쓰되, 없으면 사례 언급 생략\n"
        "5) 총 5~10문장, 과도한 반복/장황함 금지\n"
        f"{('참고 가능한 조문: ' + cite_str) if cite_str else ''}\n\n"
        "통합 초안:\n" + text
    )
    return LLM.invoke(prompt).content

def summarization_node(state: QAState) -> QAState:
    combined = state.get("combined") or ""
    if not combined.strip():
        return {**state, "summary": "I don't know"}

    have_guideline = bool(state.get("result_guideline"))
    have_law       = bool(state.get("result_law"))
    have_cases     = bool(state.get("result_case_service") or state.get("result_case_product"))
    citations      = state.get("law_citations") or []

    summary = summarize_text_prose(
        combined,
        have_guideline=have_guideline,
        have_law=have_law,
        have_cases=have_cases,
        citations=citations
    )

    return {**state, "summary": summary}

# -----------------------------------------------------------------------------
# 그래프 구성 (노드명 'summary' → 'summarize_node'로 변경)
# -----------------------------------------------------------------------------
def build_graph() -> Any:
    g = StateGraph(QAState)
    g.add_node("router", router_node)
    g.add_node("guideline", guideline_node)
    g.add_node("law",       law_node)
    g.add_node("case_service", case_service_node)
    g.add_node("case_product", case_product_node)
    g.add_node("combine", combine_node)
    g.add_node("summarize_node", summarization_node)  # ← 변경

    g.set_entry_point("router")
    g.add_edge("router", "guideline")

    # 해결기준 → (법률/사례/컴바인)
    def after_guideline(state: QAState) -> str:
        if state.get("need_law"): return "to_law"
        if state.get("need_case_service"): return "to_case_service"
        if state.get("need_case_product"): return "to_case_product"
        return "to_combine"

    g.add_conditional_edges("guideline", after_guideline, {
        "to_law": "law",
        "to_case_service": "case_service",
        "to_case_product": "case_product",
        "to_combine": "combine",
    })

    # 법률 → (사례/컴바인)
    def after_law(state: QAState) -> str:
        if state.get("need_case_service"): return "to_case_service"
        if state.get("need_case_product"): return "to_case_product"
        return "to_combine"

    g.add_conditional_edges("law", after_law, {
        "to_case_service": "case_service",
        "to_case_product": "case_product",
        "to_combine": "combine",
    })

    # 서비스 사례 → (상품 사례/컴바인)
    g.add_conditional_edges(
        "case_service",
        lambda s: "to_case_product" if s.get("need_case_product") else "to_combine",
        {"to_case_product": "case_product", "to_combine": "combine"},
    )

    # 상품 사례 → 컴바인
    g.add_edge("case_product", "combine")
    # 컴바인 → 요약 → END
    g.add_edge("combine", "summarize_node")  # ← 변경
    g.add_edge("summarize_node", END)        # ← 변경

    return g.compile(checkpointer=InMemorySaver())

# -----------------------------------------------------------------------------
# 실행 헬퍼
# -----------------------------------------------------------------------------
def run_query(graph, question: str, thread_id: str = "consumer-rag-final-prose") -> Dict[str, Any]:
    init: QAState = {
        "query": question,
        "need_guideline": True,
        "need_law": False,
        "need_case_service": False,
        "need_case_product": False,
        "result_guideline": "",
        "result_law": "",
        "result_case_service": "",
        "result_case_product": "",
        "law_citations": [],
        "combined": "",
        "summary": "",
        "source_documents_guideline": [],
        "source_documents_law": [],
        "source_documents_case_service": [],
        "source_documents_case_product": [],
        "source_documents": [],
    }
    return graph.invoke(init, config={"configurable": {"thread_id": thread_id}})


In [ ]:
# 단일 질문 실행 코드 (build_graph(), run_query()가 이미 정의되어 있어야 함)

# 1) 그래프 빌드
graph = build_graph()

# 2) 질문 설정
question = "해외 직구 전자제품이 초기불량일 때 교환/환불은 어떻게 되나요? 배송비는 누가 부담하나요?"

# 3) 실행
state = run_query(graph, question=question, thread_id="single-run-1")

# 4) 결과 출력
print("[질문]")
print(state["query"])

print("\n[최종 요약(미괄식 한 문단)]")
print(state.get("summary", ""))

print("\n[참고 조문 자동추출]")
print(state.get("law_citations", []))

[질문]
해외 직구 전자제품이 초기불량일 때 교환/환불은 어떻게 되나요? 배송비는 누가 부담하나요?

[최종 요약(미괄식 한 문단)]
해외 직구 전자제품이 초기불량인 경우, 소비자는 판매자에게 교환 또는 환불을 요청할 수 있으며, 이때 발생하는 배송비는 일반적으로 판매자가 부담하는 것이 원칙입니다. 전자상거래 등에서의 소비자보호에 관한 법률에 따르면, 소비자는 재화의 내용이 광고와 다르거나 계약 내용과 다르게 이행된 경우 청약철회를 할 수 있습니다. 또한, 소비자가 청약철회를 한 경우에는 이미 공급받은 재화를 반환해야 하며, 판매자는 소비자에게 위약금이나 손해배상을 청구할 수 없습니다. 품질보증기간 이내에 하자가 발생한 경우, 소비자는 제품 교환 또는 구입가 환급을 요청할 수 있으며, 품질보증기간이 경과한 경우에는 유상수리 후 교환이 가능합니다. 따라서 소비자는 초기불량을 증명할 수 있는 자료를 준비하여 판매자에게 요청해야 하며, 판매자가 요구하는 절차에 따라 진행해야 합니다. 이러한 절차를 통해 소비자는 자신의 권리를 보호받을 수 있습니다.

[참고 조문 자동추출]
['제17조 제1항', '제17조 제3항', '제19조', '제18조']
